In [6]:
%pip install tensorflow keras opencv-python scikit-image matplotlib seaborn numpy

^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np
from skimage import exposure

def load_and_preprocess_image(path, size=(256, 256)):
    """Tải và tiền xử lý ảnh X-quang."""
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Đọc ảnh grayscale
    img = cv2.resize(img, size)  # Chuẩn hóa kích thước
    img = exposure.equalize_adapthist(img, clip_limit=0.03)  # Tăng cường độ tương phản bằng CLAHE
    img = img / 255.0  # Chuẩn hóa giá trị pixel về [0,1]
    return img

# Ví dụ sử dụng
if __name__ == "__main__":
    sample_img = load_and_preprocess_image("data/train/NORMAL/IM-0115-0001.jpeg")
    cv2.imwrite("preprocessed_sample.jpg", (sample_img * 255).astype(np.uint8))

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose

def build_denoising_autoencoder(input_shape=(256, 256, 1)):
    """Xây dựng mô hình Denoising Autoencoder."""
    inputs = Input(shape=input_shape)
    
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    # Decoder
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = Model(inputs, decoded)
    return autoencoder

# Ví dụ sử dụng
if __name__ == "__main__":
    model = build_denoising_autoencoder()
    model.summary()  # In cấu trúc mô hình

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from model import build_denoising_autoencoder
import os

# Đường dẫn đến dữ liệu
train_dir = "data/train"
val_dir = "data/val"

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

# Load data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input'  # Dùng cho autoencoder
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input'
)

# Build và compile model
model = build_denoising_autoencoder()
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Huấn luyện
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Lưu mô hình
model.save("denoising_autoencoder.h5")

# Visualize loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig("training_loss.png")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import cv2
import numpy as np
from data_preprocessing import load_and_preprocess_image

# Load mô hình
model = load_model("denoising_autoencoder.h5")

# Load ảnh test mẫu
test_img_path = "data/test/NORMAL/IM-0001-0001.jpeg"
test_img = load_and_preprocess_image(test_img_path)
test_img_input = np.expand_dims(test_img, axis=0)  # Thêm batch dimension

# Dự đoán
predicted_img = model.predict(test_img_input)[0]

# Tính PSNR và SSIM
psnr = peak_signal_noise_ratio(test_img, predicted_img)
ssim = structural_similarity(test_img, predicted_img, multichannel=False)

print(f"PSNR: {psnr:.2f}")
print(f"SSIM: {ssim:.2f}")

# Visualize
import matplotlib.pyplot as plt
plt.subplot(1, 2, 1)
plt.imshow(test_img, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(predicted_img, cmap='gray')
plt.title('Denoised Image')
plt.axis('off')
plt.savefig("comparison.png")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
from data_preprocessing import load_and_preprocess_image
import cv2
import numpy as np

# Load mô hình
model = load_model("denoising_autoencoder.h5")

# Load ảnh mới
new_img_path = "data/test/PNEUMONIA/person1_virus_6.jpeg"  # Thay bằng đường dẫn ảnh của bạn
new_img = load_and_preprocess_image(new_img_path)
new_img_input = np.expand_dims(new_img, axis=0)

# Dự đoán
denoised_img = model.predict(new_img_input)[0]

# Lưu ảnh cải thiện
cv2.imwrite("denoised_image.jpg", (denoised_img * 255).astype(np.uint8))

# Hiển thị kết quả
import matplotlib.pyplot as plt
plt.subplot(1, 2, 1)
plt.imshow(new_img, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(denoised_img, cmap='gray')
plt.title('Denoised Image')
plt.axis('off')
plt.show()